In [1]:
import tensorflow as tf
import numpy as np
import wavio
import os
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [11]:
# data preprocessing
seq_length = 2000
merge_factor = 4#temporal resolution = merge_factor/16000 #discrimination_window = merge_factor*seq_length/16000
n_train = 225; n_test = 225;
num_channels = 2
input_size = num_channels*merge_factor #n_channels*merge_factor
num_classes = 2 #n_directions
offset = 0
train_x = []; train_t = []
test_x = []; test_t = []
for i in range(num_classes):
    angle = 0+(1*i)
    temp_data = wavio.read('at1/'+'d'+str(angle)+'.WAV').data
    for j in range(n_train+n_test):
        if j%2 != 1:
            train_x.append(temp_data[offset+(j*seq_length*merge_factor):offset+((j+1)*seq_length*merge_factor)])
            train_t.append(i)
        else:
            test_x.append(temp_data[offset+(j*seq_length*merge_factor):offset+((j+1)*seq_length*merge_factor)])
            test_t.append(i)

train_x = np.array(train_x); train_t = np.array(train_t)
train_x = train_x.reshape(n_train*num_classes,seq_length,input_size)
mean = np.mean(train_x); std = np.std(train_x)
#train_x = (train_x - mean + 0.0)/std
test_x = np.array(test_x); test_t = np.array(test_t)
test_x = test_x.reshape(n_test*num_classes,seq_length,input_size)
#test_x = (test_x - mean + 0.0)/std

In [12]:
x = tf.placeholder(tf.float32, [None,seq_length, input_size])
labels = tf.placeholder(tf.int32, [None])

In [13]:
# hyperparameters
context_size = 40
learning_rate = tf.placeholder(tf.float32,[])

In [14]:
# parameters
r_i = np.sqrt(6.0/(input_size + context_size))
r_c = np.sqrt(6.0/(context_size + context_size))
r_y = np.sqrt(6.0/(context_size + num_classes))
params = {
    'Wfx': tf.Variable(tf.random_uniform([input_size, context_size],-r_i,r_i)),
    'Wfc': tf.Variable(tf.random_uniform([context_size, context_size],-r_c,r_c)),
    'Wsx': tf.Variable(tf.random_uniform([input_size, context_size],-4*r_i,4*r_i)),
    'Wsc': tf.Variable(tf.random_uniform([context_size, context_size],-4*r_c,4*r_c)),
    'Wyc': tf.Variable(tf.random_normal([context_size,num_classes],stddev=r_y)),
    'bf': tf.Variable(1.0*tf.ones([context_size])),
    'bs': tf.Variable(tf.random_normal([context_size],stddev=0.01)),
    'by': tf.Variable(tf.zeros([num_classes]))
}

In [15]:
# parameters
sdev=0.1
params = {
    'Wfx': tf.Variable(tf.random_normal([input_size, context_size],stddev=sdev)),
    'Wfc': tf.Variable(tf.random_normal([context_size, context_size],stddev=sdev)),
    'Wsx': tf.Variable(tf.random_normal([input_size, context_size],stddev=sdev)),
    'Wsc': tf.Variable(tf.random_normal([context_size, context_size],stddev=sdev)),
    'Wyc': tf.Variable(tf.random_normal([context_size,num_classes],stddev=sdev)),
    'bf': tf.Variable(2*tf.ones([context_size])),
    'bs': tf.Variable(tf.random_normal([context_size],stddev=sdev)),
    'by': tf.Variable(tf.zeros([num_classes]))
}

In [16]:
# architecture
def LSTM(x,params):
    batch_size = tf.shape(x)[0]
    c = tf.zeros([batch_size,context_size])
    for t in range(seq_length):
        f = tf.sigmoid(tf.add(tf.add(tf.matmul(x[:,t,:],params['Wfx']),tf.matmul(c,params['Wfc'])),params['bf']))
        s = tf.tanh(tf.add(tf.add(tf.matmul(x[:,t,:],params['Wsx']),tf.matmul(c,params['Wsc'])),params['bs']))
        c = tf.add(tf.mul(f,c),tf.mul(1-f,s))
    y = tf.add(tf.matmul(c,params['Wyc']),params['by'])
    return y

In [17]:
# training strategy
y = LSTM(x,params)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(y,labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
#optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [18]:
sess.run(tf.initialize_all_variables())

In [20]:
n_iter = 0
while True:
    _,cost_ = sess.run([optimizer,cost],feed_dict={x:train_x,labels:train_t,learning_rate:0.01})
    print('iter'+str(n_iter)+' cost:'+str(cost_))
    if n_iter % 10 == 0:
        y_train = sess.run(y,feed_dict={x:train_x})
        print 'train error = ' + str((sum(np.argmax(y_train,axis=1) == train_t) + 0.0)/(n_train*num_classes))
        y_test = sess.run(y,feed_dict={x:test_x})
        print 'test error = ' + str((sum(np.argmax(y_test,axis=1) == test_t) + 0.0)/(n_test*num_classes))+'\n'
#     if n_iter % 100 == 0:
#         mypath = 'vars/iter'+str(n_iter)
#         if not os.path.isdir(mypath):
#            os.makedirs(mypath)
#         myfile = mypath+'/'+str(n_iter)
#         saver.save(sess,myfile,write_meta_graph=False)
    n_iter += 1

iter0 cost:0.647022
train error = 0.615555555556
test error = 0.551111111111

iter1 cost:0.647017
iter2 cost:0.647012
iter3 cost:0.647007
iter4 cost:0.647002
iter5 cost:0.646997
iter6 cost:0.646991
iter7 cost:0.646986
iter8 cost:0.646981
iter9 cost:0.646976
iter10 cost:0.646971
train error = 0.617777777778
test error = 0.548888888889

iter11 cost:0.646966
iter12 cost:0.646961
iter13 cost:0.646956
iter14 cost:0.646951
iter15 cost:0.646946
iter16 cost:0.646941
iter17 cost:0.646936
iter18 cost:0.646931
iter19 cost:0.646926
iter20 cost:0.646921
train error = 0.617777777778
test error = 0.548888888889

iter21 cost:0.646916
iter22 cost:0.646911
iter23 cost:0.646906
iter24 cost:0.646901
iter25 cost:0.646896
iter26 cost:0.646891
iter27 cost:0.646886
iter28 cost:0.646881
iter29 cost:0.646876
iter30 cost:0.646871
train error = 0.617777777778
test error = 0.546666666667

iter31 cost:0.646866
iter32 cost:0.646861
iter33 cost:0.646856
iter34 cost:0.646852
iter35 cost:0.646847
iter36 cost:0.646842
i

KeyboardInterrupt: 

In [9]:
sess.close()